In [1]:
%run "../head.py"

In [2]:
import sys
import concurrent.futures
from pathlib import Path

import numpy as np
import pandas as pd
from hic_basic.hicio import read_meta, load_pickle, load_json
from lib.struct import pileup_bf, consecutive_slice_pileup_bf, pileup_bf_mt

In [3]:
# h.dsdir = Path("/shareb/ychi/repo/sperm_struct/ds_pipeline/smk")
# Sdir = Path("/share/home/ychi/dev/sperm_struct")
# h.ddir = Path("/shareb/ychi/repo/sperm_struct/notebooks/data/")
if h.version in ["0","1"]:
    meta = read_meta("../A_meta/meta/tillsperm55.meta.csv.gz")
    hg_meta = read_meta("../A_meta/meta/tillsperm62_hg.meta.csv.gz")
    GM_meta = read_meta("../A_meta/meta/tillsperm50_GM.meta.csv.gz")
    mESC_meta = read_meta("../A_meta/meta/tillsperm56_mESC.meta.csv.gz")
    Tan2018_meta = read_meta("../A_meta/meta/Tan2018.meta.csv.gz")
    RS_meta = read_meta(h.sdir / "ds_pipeline/smk/config" / "RS.gs.sample_table.csv")
    meta = pd.concat([meta, hg_meta, GM_meta, mESC_meta, Tan2018_meta, RS_meta],axis=0)
elif h.version == "2":
    meta = read_meta("../A_meta/meta/All_formal.meta.csv.gz")
if h.version in ["0","1"]:
    primary_views = load_pickle(h.dsdir / "primary_views/PFA05.64.pkl")
    primary_views.update(load_pickle(h.dsdir / "primary_views/P3.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/GM.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/Sperm.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/Sperm_hg.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/mESC.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/Tan2018.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/RS.64.pkl"))
elif h.version == "2":
    primary_views = {}
    primary_views.update(load_pickle(h.dsdir / "primary_views/Sperm.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/Sperm_hg.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/mESC.64.pkl"))
    primary_views.update(load_pickle(h.dsdir / "primary_views/Tan2018.64.pkl"))

In [4]:
if h.version in ["0","1"]:
    GM_samples = list(load_pickle(h.dsdir / "primary_views/GM.64.pkl").keys())
    RS_samples = RS_meta.index.tolist()
mESC_samples = load_json(h.fig1 / f"Fig1S_tech_analysis/mESC_samples_with_structure.v{h.version}.json")
Tan2018_samples = load_json(h.fig1 / f"Fig1S_tech_analysis/Tan2018_samples_with_structure.v{h.version}.json")


Sperm_targets = pd.read_csv(
    f"../B_auto_alignment/Sperm.round1.targets.v{h.version}.csv",
    index_col = 0,
)
Sperm_targets.columns = ["ht", "dv", "lr"]
Sperm_hg_targets = pd.read_csv(
    f"../B_auto_alignment/Sperm_hg.round1.targets.v{h.version}.csv",
    index_col = 0,
)
Sperm_hg_targets.columns = ["ht", "dv", "lr"]
mESC_fake_targets = pd.DataFrame(
        [[1,1,1] for _ in mESC_samples],
        index = mESC_samples,
        columns = ["ht", "dv", "lr"] 
        ) # fake targets for exploratory purpose
Tan2018_fake_targets = pd.DataFrame(
        [[1,1,1] for _ in Tan2018_samples],
        index = Tan2018_samples,
        columns = ["ht", "dv", "lr"] 
        ) # fake targets for exploratory purpose
if h.version in ["0","1"]:
    GM_fake_targets = pd.DataFrame(
            [[1,1,1] for _ in GM_samples],
            index = GM_samples,
            columns = ["ht", "dv", "lr"] 
            ) # fake targets for exploratory purpose
    RS_fake_targets = pd.DataFrame(
            [[1,1,1] for _ in RS_samples],
            index = RS_samples,
            columns = ["ht", "dv", "lr"] 
            ) # fake targets for exploratory purpose
    targets = pd.concat(
        [
            Sperm_targets, Sperm_hg_targets, 
            GM_fake_targets, mESC_fake_targets, Tan2018_fake_targets,
            RS_fake_targets
            ]
            )
elif h.version == "2":
    targets = pd.concat(
        [Sperm_targets, Sperm_hg_targets, mESC_fake_targets, Tan2018_fake_targets]
    )

#Sperm_samples = load_json(f"../Fig1S_tech_analysis/Sperm_samples_with_structure.v{h.version}.json")
Sperm_samples = load_json(h.fig1 / f"Fig1S_structure_quality/Sperm_ga.v{h.version}.json")
mouse1_samples_mask = (meta["biosample_identity"] == "mouse1") & (meta.index.isin(Sperm_samples))
mouse1_samples = meta[mouse1_samples_mask].index.tolist()
mouse2_samples_mask = (meta["biosample_identity"] == "mouse2") & (meta.index.isin(Sperm_samples))
mouse2_samples = meta[mouse2_samples_mask].index.tolist()
#Sperm_hg_samples = load_json(f"../Fig1S_tech_analysis/Sperm_hg_samples_with_structure.v{h.version}.json")
Sperm_hg_samples = load_json(h.fig1 / f"Fig1S_structure_quality/Sperm_hg_ga.v{h.version}.json")
Sperm_Zheng_mask = (meta["biosample_identity"] == "Zheng") & (meta.index.isin(Sperm_hg_samples))
Sperm_Zheng_samples = meta[Sperm_Zheng_mask].index.tolist()
Sperm_Liu_mask = (meta["biosample_identity"] == "Liu") & (meta.index.isin(Sperm_hg_samples))
Sperm_Liu_samples = meta[Sperm_Liu_mask].index.tolist()
Sperm_X_bearing_samples = load_json(h.fig1 / f"Fig1S_XY_ratio/X_sperm.v{h.version}.json")
Sperm_Y_bearing_samples = load_json(h.fig1 / f"Fig1S_XY_ratio/Y_sperm.v{h.version}.json")

### Calculate primary views

Done in ds_pipelin/direct/Primary_views.sh.

### Prepare arguments for pileup_bf

In [5]:
if h.version in ["0","1"]:
    batches = [
        "Sperm",
        "mouse1",
        "mouse2",
        "Sperm_hg",
        "Sperm_Liu",
        "Sperm_Zheng",
        "GM",
        "Sperm_X_bearing",
        "Sperm_Y_bearing",
        "mESC",
        "Tan2018",
        "RS"
    ]
    arg_targetses = { # batch_name : target df
        "Sperm" : targets.loc[Sperm_samples],
        "mouse1" : targets.loc[mouse1_samples],
        "mouse2" : targets.loc[mouse2_samples],
        "Sperm_hg" : targets.loc[Sperm_hg_samples],
        "Sperm_Liu" : targets.loc[Sperm_Liu_samples],
        "Sperm_Zheng" : targets.loc[Sperm_Zheng_samples],
        "GM" : targets.loc[GM_samples],
        "Sperm_X_bearing" : targets.loc[Sperm_X_bearing_samples],
        "Sperm_Y_bearing" : targets.loc[Sperm_Y_bearing_samples],
        "mESC" : targets.loc[mESC_samples],
        "Tan2018" : targets.loc[Tan2018_samples],
        "RS" : targets.loc[RS_samples]
    }
    arg_featureses = { # batch_name : feature df
        "Sperm" : pd.read_csv(h.ddir / "Sperm.mm10.features.csv.gz", index_col = [0,1]),
        "mouse1" : pd.read_csv(h.ddir / "Sperm.mm10.features.csv.gz", index_col = [0,1]),
        "mouse2" : pd.read_csv(h.ddir / "Sperm.mm10.features.csv.gz", index_col = [0,1]),
        "Sperm_hg" : pd.read_csv(h.ddir / "Sperm_hg.GRCh38.features.csv.gz", index_col = [0,1]),
        "Sperm_Liu" : pd.read_csv(h.ddir / "Sperm_hg.GRCh38.features.csv.gz", index_col = [0,1]),
        "Sperm_Zheng" : pd.read_csv(h.ddir / "Sperm_hg.GRCh38.features.csv.gz", index_col = [0,1]),
        "GM" : pd.read_csv(h.ddir / "hg19_dip.features.csv.gz", index_col = [0,1]),
        "Sperm_X_bearing" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1]),
        "Sperm_Y_bearing" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1]),
        "mESC" : pd.read_csv(h.ddir / "mm10_dip.features.csv.gz", index_col = [0,1]),
        "Tan2018" : pd.read_csv(h.ddir / "GRCh38_dip.features.csv.gz", index_col = [0,1]),
        "RS" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1])
    }
    arg_aggses = {
        "Sperm" : load_pickle(h.ddir / "Sperm.mm10.aggs.pkl"),
        "mouse1" : load_pickle(h.ddir / "Sperm.mm10.aggs.pkl"),
        "mouse2" : load_pickle(h.ddir / "Sperm.mm10.aggs.pkl"),
        "Sperm_hg" : load_pickle(h.ddir / "Sperm_hg.GRCh38.aggs.pkl"),
        "Sperm_Liu" : load_pickle(h.ddir / "Sperm_hg.GRCh38.aggs.pkl"),
        "Sperm_Zheng" : load_pickle(h.ddir / "Sperm_hg.GRCh38.aggs.pkl"),
        "GM" : load_pickle(h.ddir / "hg19_dip.aggs.pkl"),
        "Sperm_X_bearing" : load_pickle(h.ddir / "mm10.aggs.pkl"),
        "Sperm_Y_bearing" : load_pickle(h.ddir / "mm10.aggs.pkl"),
        "mESC" : load_pickle(h.ddir / "mm10_dip.aggs.pkl"),
        "Tan2018" : load_pickle(h.ddir / "GRCh38_dip.aggs.pkl"),
        "RS" : load_pickle(h.ddir / "mm10.aggs.pkl")
    }
elif h.version == "2":
    batches = ["Sperm","Sperm_hg","mESC","Tan2018"]
    arg_sampleses = {
        "Sperm" : Sperm_samples,
        "Sperm_hg" : Sperm_hg_samples,
        "mESC" : mESC_samples,
        "Tan2018" : Tan2018_samples
    }
    arg_targetses = { # batch_name : target df
        "Sperm" : targets.loc[Sperm_samples],
        #"mouse1" : targets.loc[mouse1_samples],
        #"mouse2" : targets.loc[mouse2_samples],
        "Sperm_hg" : targets.loc[Sperm_hg_samples],
        #"Sperm_Liu" : targets.loc[Sperm_Liu_samples],
        #"Sperm_Zheng" : targets.loc[Sperm_Zheng_samples],
        #"GM" : targets.loc[GM_samples],
        #"Sperm_X_bearing" : targets.loc[Sperm_X_bearing_samples],
        #"Sperm_Y_bearing" : targets.loc[Sperm_Y_bearing_samples],
        "mESC" : targets.loc[mESC_samples],
        "Tan2018" : targets.loc[Tan2018_samples],
        #"RS" : targets.loc[RS_samples]
    }
    arg_featureses = { # batch_name : feature df
        "Sperm" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1]),
        #"mouse1" : pd.read_csv(h.ddir / "Sperm.mm10.features.csv.gz", index_col = [0,1]),
        #"mouse2" : pd.read_csv(h.ddir / "Sperm.mm10.features.csv.gz", index_col = [0,1]),
        "Sperm_hg" : pd.read_csv(h.ddir / "GRCh38.features.csv.gz", index_col = [0,1]),
        #"Sperm_Liu" : pd.read_csv(h.ddir / "Sperm_hg.GRCh38.features.csv.gz", index_col = [0,1]),
        #"Sperm_Zheng" : pd.read_csv(h.ddir / "Sperm_hg.GRCh38.features.csv.gz", index_col = [0,1]),
        #"GM" : pd.read_csv(h.ddir / "hg19_dip.features.csv.gz", index_col = [0,1]),
        #"Sperm_X_bearing" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1]),
        #"Sperm_Y_bearing" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1]),
        "mESC" : pd.read_csv(h.ddir / "mm10_dip.features.csv.gz", index_col = [0,1]),
        "Tan2018" : pd.read_csv(h.ddir / "GRCh38_dip.features.csv.gz", index_col = [0,1]),
        #"RS" : pd.read_csv(h.ddir / "mm10.features.csv.gz", index_col = [0,1])
    }
    arg_aggses = {
        "Sperm" : load_pickle(h.ddir / "mm10.aggs.pkl"),
        #"mouse1" : load_pickle(h.ddir / "Sperm.mm10.aggs.pkl"),
        #"mouse2" : load_pickle(h.ddir / "Sperm.mm10.aggs.pkl"),
        "Sperm_hg" : load_pickle(h.ddir / "GRCh38.aggs.pkl"),
        #"Sperm_Liu" : load_pickle(h.ddir / "Sperm_hg.GRCh38.aggs.pkl"),
        #"Sperm_Zheng" : load_pickle(h.ddir / "Sperm_hg.GRCh38.aggs.pkl"),
        #"GM" : load_pickle(h.ddir / "hg19_dip.aggs.pkl"),
        #"Sperm_X_bearing" : load_pickle(h.ddir / "mm10.aggs.pkl"),
        #"Sperm_Y_bearing" : load_pickle(h.ddir / "mm10.aggs.pkl"),
        "mESC" : load_pickle(h.ddir / "mm10_dip.aggs.pkl"),
        "Tan2018" : load_pickle(h.ddir / "GRCh38_dip.aggs.pkl"),
        #"RS" : load_pickle(h.ddir / "mm10.aggs.pkl")
    }

arg_bfs_lr_ss_files = { # single slice
    batch : (h.ddir / batch).with_suffix(".lr_ss.bfs.pkl")
    for batch in batches
}

arg_bfs_dv_ss_files = {
    batch : (h.ddir / batch).with_suffix(".dv_ss.bfs.pkl")
    for batch in batches
}

arg_bfs_lr_ft_files = { # full thickness
    batch : (h.ddir / batch).with_suffix(".lr_ft.bfs.pkl")
    for batch in batches
}

arg_bfs_dv_ft_files = {
    batch : (h.ddir / batch).with_suffix(".dv_ft.bfs.pkl")
    for batch in batches
}

# 3-grouper
arg_bfs_vx_files = {
    batch : (h.ddir / batch).with_suffix(".vx.bfs.h5")
    for batch in batches
}

arg_bfs_lr_cs_files = { # consecutive slice
    batch : (h.ddir / batch).with_suffix(".lr_cs.bfs.pkl")
    for batch in batches
}

arg_bfs_dv_cs_files = {
    batch : (h.ddir / batch).with_suffix(".dv_cs.bfs.pkl")
    for batch in batches
}

### Do the pileups

3-grouper

In [ ]:
Fin = False
Force = False
outfiles = arg_bfs_vx_files
def process_batch(batch, **kwargs):
    samples, targets, bf_file = \
        arg_sampleses[batch], arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force and  not bf_file.is_dir():
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 2,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        min_particles={"density" : 0},
        **kwargs
    )
    print(f"{bf_file} done")
    return bfs

human sperm

In [ ]:
scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"

#arg_aggses["Sperm_hg"].pop("scAB_dist")
arg_aggses["Sperm_hg"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm_hg"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["Sperm_hg"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
process_batch(
    "Sperm_hg",
    pb_features = {
        "scAB" : dict(zip(
            arg_sampleses["Sperm_hg"],
            [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm_hg"]]
            )),
        "scIS" : dict(zip(
            arg_sampleses["Sperm_hg"],
            [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm_hg"]]
            )),
        "scAB_rank_normed" : dict(zip(
            arg_sampleses["Sperm_hg"],
            [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses["Sperm_hg"]]
            ))
    },
    # pb_readers = {
    #     "scAB" : lambda filepath : pd.read_table(
    #         filepath,
    #         names = ["chr","pos","scAB"],
    #         index_col = [0,1]
    #         )["scAB"]
    # },
    n_threads = 16
    )

human autosomes

In [ ]:
scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
#arg_aggses["Sperm_hg"].pop("scAB_dist")
arg_aggses["Sperm_hg"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm_hg"].update({"scIS_dist" : ("scIS","mean")})

batch = "Sperm_hg"
outfile = h.ddir / "Sperm_hg_autosomes.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 2,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm_hg"]]
                )),
            "scIS" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm_hg"]]
                ))
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

human 1-cube

In [8]:
Force=True

scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
sc_intermingle_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/intermingle"
#arg_aggses["Sperm_hg"].pop("scAB_dist")
arg_aggses["Sperm_hg"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm_hg"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["Sperm_hg"].update({"sum_CpG" : ("CpG", "sum")})
arg_aggses["Sperm_hg"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
arg_aggses["Sperm_hg"].update({"sum_scAB_rank_normed" : ("scAB_rank_normed","sum")})
arg_aggses["Sperm_hg"].update({"mean_intermingle" : ("sc_intermingle","mean")})

batch = "Sperm_hg"
outfile = h.ddir / "Sperm_hg_1cube.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
samples = ["HuS07_HuS1L514", "HuS02_HuSZ049", "HuS02_HuSZ194", "HuS02_HuSZ350", "HuS02_HuSZ236",
    "HuS03_HuSZ437", "HuS02_HuSZ069", "HuS03_HuSZ452", "HuS02_HuSZ352","HuS06_HuS1L113","HuS02_HuSZ330"]
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 1,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        #sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        global_min_particle = 1,
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm_hg"]]
                )),
            "scIS" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm_hg"]]
                )),
            "scAB_rank_normed" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses["Sperm_hg"]]
                )),
            "sc_intermingle" : dict(zip(
                arg_sampleses["Sperm_hg"],
                [str(sc_intermingle_dir+f"/{sample}.R3.multi_chrom_intermingling.ref.tsv.gz") for sample in arg_sampleses["Sperm_hg"]]
                )),
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

Force = False

Processing cells: 100%|██████████| 11/11 [00:25<00:00,  2.36s/it]


Combining results...


Writing to file: 100%|██████████| 11/11 [01:01<00:00,  5.56s/it]


Indexing...
All done.
/shareb/ychi/repo/sperm_struct/notebooks/data2/Sperm_hg_1cube.vx.bfs.h5 done


In [ ]:
process_batch(
    "Tan2018",
    n_threads = 16,
    global_min_particle = 0
    )

Tan2018 1-cube

In [9]:
# arg_aggses["Tan2018"].pop("scAB_rank_normed_dist")
arg_aggses["Tan2018"].pop("sum_scAB_rank_normed")

('scAB_rank_normed', 'sum')

In [10]:
Force=True

scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
sc_intermingle_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/intermingle"
#arg_aggses["Tan2018"].pop("scAB_dist")
arg_aggses["Tan2018"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Tan2018"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["Tan2018"].update({"sum_CpG" : ("CpG", "sum")})
# arg_aggses["Tan2018"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
# arg_aggses["Tan2018"].update({"sum_scAB_rank_normed" : ("scAB_rank_normed","sum")})
arg_aggses["Tan2018"].update({"mean_intermingle" : ("sc_intermingle","mean")})

batch = "Tan2018"
outfile = h.ddir / "Tan2018_1cube.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 1,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        #sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        global_min_particle = 1,
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses["Tan2018"],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Tan2018"]]
                )),
            "scIS" : dict(zip(
                arg_sampleses["Tan2018"],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Tan2018"]]
                )),
            # "scAB_rank_normed" : dict(zip(
            #     arg_sampleses["Tan2018"],
            #     [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses["Tan2018"]]
            #     )),
            "sc_intermingle" : dict(zip(
                arg_sampleses["Tan2018"],
                [str(sc_intermingle_dir+f"/{sample}.R3.multi_chrom_intermingling.ref.tsv.gz") for sample in arg_sampleses["Tan2018"]]
                )),
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

Force = False

Processing cells:   0%|          | 0/14 [00:00<?, ?it/s]

Processing cells: 100%|██████████| 14/14 [00:53<00:00,  3.81s/it]


Combining results...


Writing to file: 100%|██████████| 14/14 [01:21<00:00,  5.83s/it]


Indexing...
All done.
/shareb/ychi/repo/sperm_struct/notebooks/data2/Tan2018_1cube.vx.bfs.h5 done


mouse sperm

In [ ]:
scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
#arg_aggses["Sperm_hg"].pop("scAB_dist")
arg_aggses["Sperm"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["Sperm"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
process_batch(
    "Sperm",
    pb_features = {
        "scAB" : dict(zip(
            arg_sampleses["Sperm"],
            [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm"]]
            )),
        "scIS" : dict(zip(
            arg_sampleses["Sperm"],
            [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm"]]
            )),
        "scAB_rank_normed" : dict(zip(
            arg_sampleses["Sperm"],
            [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses["Sperm"]]
            ))
    },
    # pb_readers = {
    #     "scAB" : lambda filepath : pd.read_table(
    #         filepath,
    #         names = ["chr","pos","scAB"],
    #         index_col = [0,1]
    #         )["scAB"]
    # },
    n_threads = 16
    )

mouse autosomes

In [ ]:
scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
arg_aggses["Sperm"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm"].update({"scIS_dist" : ("scIS","mean")})
batch = "Sperm"
outfile = h.ddir / "Sperm_autosomes.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 2,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses["Sperm"],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm"]]
                )),
            "scIS" : dict(zip(
                arg_sampleses["Sperm"],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm"]]
                ))
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

mouse sperm 1-cube

In [7]:
Force = True

scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
sc_intermingle_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/intermingle"
#arg_aggses["Sperm_hg"].pop("scAB_dist")
arg_aggses["Sperm"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["Sperm"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["Sperm"].update({"sum_CpG" : ("CpG", "sum")})
arg_aggses["Sperm"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
arg_aggses["Sperm"].update({"sum_scAB_rank_normed" : ("scAB_rank_normed","sum")})
arg_aggses["Sperm"].update({"mean_intermingle" : ("sc_intermingle","mean")})

batch = "Sperm"
outfile = h.ddir / "Sperm_1cube.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
samples = ["BJ8019","BJ9057","BJ8027","BJ8029","BJ8034","BJ8039","BJ8141","BJ8174","BJ9142","BJ8041"]
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 1,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        #sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        global_min_particle = 1,
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses["Sperm"],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses["Sperm"]]
                )),
            "scIS" : dict(zip(
                arg_sampleses["Sperm"],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses["Sperm"]]
                )),
            "scAB_rank_normed" : dict(zip(
                arg_sampleses["Sperm"],
                [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses["Sperm"]]
                )),
            "sc_intermingle" : dict(zip(
                arg_sampleses["Sperm"],
                [str(sc_intermingle_dir+f"/{sample}.R3.multi_chrom_intermingling.ref.tsv.gz") for sample in arg_sampleses["Sperm"]]
                )),
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

Force = False

Processing cells: 100%|██████████| 10/10 [00:24<00:00,  2.42s/it]


Combining results...


Writing to file: 100%|██████████| 10/10 [00:47<00:00,  4.78s/it]


Indexing...
All done.
/shareb/ychi/repo/sperm_struct/notebooks/data2/Sperm_1cube.vx.bfs.h5 done


mESC 1-cubed

In [6]:
Force=True

scAB_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scAB"
scIS_dir = "/shareb/ychi/repo/sperm_struct/ds_pipeline/smk2/scIS_schicluster"
arg_aggses["mESC"].update({"scAB_dist" : ("scAB","mean")})
arg_aggses["mESC"].update({"scIS_dist" : ("scIS","mean")})
arg_aggses["mESC"].update({"sum_CpG" : ("CpG", "sum")})
arg_aggses["mESC"].update({"scAB_rank_normed_dist" : ("scAB_rank_normed","mean")})
arg_aggses["mESC"].update({"sum_scAB_rank_normed" : ("scAB_rank_normed","sum")})
arg_aggses["mESC"].update({"mean_intermingle" : ("sc_intermingle","mean")})

batch = "mESC"
outfile = h.ddir / "mESC_1cube.vx.bfs.h5"
samples, targets, bf_file = \
    arg_sampleses[batch], arg_targetses[batch], outfile
#samples = ["HuS07_HuS1L514", "HuS02_HuSZ049", "HuS02_HuSZ194", "HuS02_HuSZ350", "HuS02_HuSZ236"]
if bf_file.exists() and not Force and  not bf_file.is_dir():
    print(f"{bf_file} exists, skipping")
else:
    bfs = pileup_bf_mt(
        samples,
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        outfile = bf_file,
        cache_dir = Path(bf_file.with_suffix(".cache")),
        step = 1,
        grouping = ["lr","ht","dv"],
        agg = arg_aggses[batch],
        #sub = 'chr not in ["chrX","chrY"]',
        min_particles={"density" : 0},
        global_min_particle = 1,
        pb_features = {
            "scAB" : dict(zip(
                arg_sampleses[batch],
                [str(scAB_dir+f"/{sample}.20k.scAB.csv.gz") for sample in arg_sampleses[batch]]
                )),
            "scIS" : dict(zip(
                arg_sampleses[batch],
                [str(scIS_dir+f"/{sample}_scIS.tsv.gz") for sample in arg_sampleses[batch]]
                )),
            "scAB_rank_normed" : dict(zip(
                arg_sampleses[batch],
                [str(scAB_dir+f"/{sample}.20k.scAB.rank_norm.csv.gz") for sample in arg_sampleses[batch]]
                )),
            "sc_intermingle" : dict(zip(
                arg_sampleses[batch],
                [str(sc_intermingle_dir+f"/{sample}.R3.multi_chrom_intermingling.ref.tsv.gz") for sample in arg_sampleses[batch]]
                )),
        },
        n_threads = 16
    )
    print(f"{bf_file} done")

Force = False

Processing cells:   0%|          | 0/46 [00:00<?, ?it/s]

Processing cells:  20%|█▉        | 9/46 [01:05<01:18,  2.12s/it]

Processing cells:  26%|██▌       | 12/46 [01:07<00:41,  1.22s/it]

Processing cells:  35%|███▍      | 16/46 [01:10<00:26,  1.14it/s]

Processing cells:  57%|█████▋    | 26/46 [02:00<00:28,  1.44s/it]

...

Processing cells:  70%|██████▉   | 32/46 [02:05<00:13,  1.05it/s]

Processing cells: 100%|██████████| 46/46 [02:49<00:00,  3.69s/it]


Combining results...


Writing to file: 100%|██████████| 46/46 [04:53<00:00,  6.38s/it]


Indexing...
All done.
/shareb/ychi/repo/sperm_struct/notebooks/data2/mESC_1cube.vx.bfs.h5 done


In [ ]:
process_batch("Sperm")

In [ ]:
batch = "Sperm_hg"
samples = arg_sampleses[batch]

pb_features = {
    "scAB" : dict(zip(
        arg_sampleses["Sperm_hg"],
        [scAB_dir+f"/{sample}.20k.scAB.csv.gz" for sample in arg_sampleses["Sperm_hg"]]
        ))
}
pb_readers = {
    "scAB" : lambda filep : pd.read_table(
        filep,
        names = ["chr","pos","scAB"],
        index_col = [0,1]
        )["scAB"]
}
agg = arg_aggses[batch]
agg.update({"scAB_dist" : ("scAB","mean")})


In [ ]:
from lib.struct import pileup_bf_mt_task

In [ ]:
pileup_bf_mt_task(
    samples[0],
    primary_views[samples[0]],
    targets.loc[samples[0]],
    meta.loc[samples[0],"20k_g_struct1"],
    arg_featureses[batch],
    "test.parquet",
    agg = agg,
    pb_features=pb_features,
    pb_readers=pb_readers,
)

Single full thickness. See through lr axis (grouping by [ht, dv]).

In [ ]:
Fin = False
Force = False
outfiles = arg_bfs_lr_ft_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        agg = arg_aggses[batch],
        sub = None,
        grouping = ["ht", "dv"]
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(8) as executor:
        executor.map(process_batch, arg_targetses)

Single full thickness. See through dv axis (grouping by [ht, lr]).

In [ ]:
Fin = False
Force = False
outfiles = arg_bfs_dv_ft_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        agg = arg_aggses[batch],
        sub = None,
        grouping = ["ht", "lr"]
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(8) as executor:
        executor.map(process_batch, arg_targetses)

Single slice on lr axis.

In [ ]:
Fin = False
Force = False
outfiles = arg_bfs_lr_ss_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        agg = arg_aggses[batch],
        sub = '(lr > -5) and (lr < 5)',
        grouping = ["ht", "dv"]
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(8) as executor:
        executor.map(process_batch, arg_targetses)

Single slice on dv axis.

In [ ]:
Fin = False
Force = False
outfiles = arg_bfs_dv_ss_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        agg = arg_aggses[batch],
        sub = '(dv > -5) and (dv < 5)',
        grouping = ["ht", "lr"]
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(8) as executor:
        executor.map(process_batch, arg_targetses)

Consecutive cross section (in-sample parallel)

In [ ]:
from lib.struct import consecutive_slice_pileup_bf_parallel
Fin = False
Force = False
outfiles = arg_bfs_lr_cs_files
def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force and  not bf_file.is_dir():
        print(f"{bf_file} exists, skipping")
        return
    bfs = consecutive_slice_pileup_bf_parallel(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        step = 2,
        #step=4,
        upper = 16,
        #upper=20,
        lower = -16,
        #lower=-20,
        axis = "lr",
        agg = arg_aggses[batch],
        min_particles={"density" : 0},
        prefix = bf_file,
        threads=4
    )
    print(f"{bf_file} done")
    return 0
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

In [ ]:
if not Fin:
    process_batch("Sperm_hg")

In [ ]:
if not Fin:
    process_batch("Sperm")

Consecutive cross section (between-sample parallel)

In [ ]:
Fin = True
Force = False
outfiles = arg_bfs_lr_cs_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = consecutive_slice_pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        #step = 2,
        step=4,
        #upper = 16,
        upper=20,
        #lower = -16,
        lower=-20,
        axis = "lr",
        agg = arg_aggses[batch],
        min_particles={"density" : 0}
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(12) as executor:
        executor.map(process_batch, arg_targetses)

Consecutive ht-lr cross sections.

In [ ]:
Fin = True
Force = False
outfiles = arg_bfs_dv_cs_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = consecutive_slice_pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        step = 2,
        upper = 16,
        lower = -16,
        axis = "dv",
        agg = arg_aggses[batch],
        min_particles={"density" : 0}
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

if not Fin:
    with concurrent.futures.ProcessPoolExecutor(12) as executor:
        executor.map(process_batch, arg_targetses)

Full voxels

In [ ]:
subsets = ["Sperm","Sperm_hg"]

arg_bfs_voxel_files = {
    batch : (h.ddir / batch).with_suffix(".voxel.bfs.pkl")
    for batch in subsets
}

Fin = True
Force = False
outfiles = arg_bfs_voxel_files

def process_batch(batch):
    targets, bf_file = arg_targetses[batch], outfiles[batch]
    print(bf_file)
    if bf_file.exists() and not Force:
        print(f"{bf_file} exists, skipping")
        return
    bfs = pileup_bf(
        primary_views,
        targets,
        meta,
        features = arg_featureses[batch],
        agg = arg_aggses[batch],
        grouping = ["ht","dv","lr"],
        step=6
    )
    bfs = bfs.swaplevel(0,1,axis=1)
    bfs.to_pickle(bf_file)
    print(f"{bf_file} done")

# if not Fin:
#     with concurrent.futures.ProcessPoolExecutor(8) as executor:
#         executor.map(process_batch, subsets)

In [ ]:
#process_batch("Sperm_hg")